In [64]:

import numpy as np
import open3d as o3d
import pathlib
import math
import itertools
import matplotlib.pyplot as plt

## Loading in Point cloud data

In [111]:
# Load ply file
path = "./data/P002 2022-01-25 01_24_48.ply"

if pathlib.Path(path).exists():
    pcd = o3d.io.read_point_cloud(path)
    print('PLY file loaded')
else:
    raise FileNotFoundError("File not found.")

print('Shape of points', np.asarray(pcd.points).shape)
print('Shape of colors', np.asarray(pcd.colors).shape)

PLY file loaded
Shape of points (251759, 3)
Shape of colors (251759, 3)


In [95]:
o3d.visualization.draw_geometries([pcd])

# Preprocessing

In [109]:
import matplotlib.pyplot as plt

def viewClustersViaColours(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Test function to visualise the clusters by colours
    '''
    
    labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10, print_progress=True))
    max_label = labels.max()
    print(f"point cloud has {max_label + 1} clusters")
    colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
    colors[labels < 0] = 0
    pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])
    o3d.visualization.draw_geometries([pcd])

viewClustersViaColours(pcd)

point cloud has 17 clusters


In [112]:
from collections import Counter

def isolateLargestCluster(pcd: o3d.cpu.pybind.geometry.PointCloud, labels: np.ndarray) -> None:
    '''
    Uses DBSCAN to group and isolate the largest point cloud 
       
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
        labels (np.ndarray): Array of clusters identified by labels
    
    Returns:
        type: o3d.cpu.pybind.geometry.PointCloud
    '''

    # finding the label of largest cluster + ignoring noisy points labeled -1
    counts = Counter(labels)
    largest_cluster_label = max(counts.items(), key=lambda x: x[1] if x[0] != -1 else -1)[0]

    # get all pts / colors of largest cluster
    largest_cluster_points = np.array(pcd.points)[labels == largest_cluster_label]
    largest_cluster_colors = np.array(pcd.colors)[labels == largest_cluster_label]
    
    # new point cloud from the largest cluster pts w/ colors
    largest_cluster_pcd = o3d.geometry.PointCloud()
    largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
    largest_cluster_pcd.colors = o3d.utility.Vector3dVector(largest_cluster_colors)
    
    return largest_cluster_pcd
    
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=10, print_progress=True))
pcd = isolateLargestCluster(pcd, labels)
o3d.visualization.draw_geometries([pcd])

In [ ]:
def bounding_box_minor(self, pcd):
    # Create bounding box:
    bounds = [[-math.inf, math.inf], [-math.inf, math.inf], [0, 0.0458]]  # set the bounds
    
    bounding_box_points = list(itertools.product(*bounds))  # create limit points
    bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
        o3d.utility.Vector3dVector(bounding_box_points))  # create bounding box object
    
    # Crop the point cloud using the bounding box:
    pcd_croped = pcd.crop(bounding_box)
    dists = np.asarray(pcd.compute_point_cloud_distance(pcd_croped))
    indices = np.where(dists > 0.000001)[0]
    pcd_cropped_inv = pcd.select_by_index(indices)

    # Display the cropped point cloud:
    # self.visualise(pcd_croped, windowName="pcd cropped")
    self.visualise(pcd_cropped_inv, windowName="pcd cropped inverse")
    
    return pcd_cropped_inv

In [114]:
type(largest_cluster_pcd)

open3d.cpu.pybind.geometry.PointCloud

In [117]:
def downsample_clean(pcd: o3d.cpu.pybind.geometry.PointCloud) -> None:
    '''
    Performs downsampling, outlier removal and bounding box removal
    
    Args:
        pcd (o3d.cpu.pybind.geometry.PointCloud): PointCloud object
    
    Returns:
        type: None 
    '''

    # voxel downsampling: reducing overall num of pts
    voxel_size = 0.02
    downsampled = pcd.voxel_down_sample(voxel_size) 
    
    # outlier cleaning
    _, ind = downsampled.remove_statistical_outlier(nb_neighbors=40, std_ratio=2.0)
    cleaned_pcd = downsampled.select_by_index(ind)
    # o3d.visualization.draw_geometries([cleaned_pcd])
    
    # minimal bounding box
    pcd = 
    
    # removing majority floor
    # _, inliers = cleaned_pcd.segment_plane(distance_threshold=0.05, ransac_n=5, num_iterations=10)
    # inlier_cloud = cleaned_pcd.select_by_index(inliers)
    # inlier_cloud.paint_uniform_color([1,0,0])
    # outlier_cloud = cleaned_pcd.select_by_index(inliers, invert=True)
    # o3d.visualization.draw_geometries([outlier_cloud])
    
    # # floor cleaning
    # pcd = self.bounding_box_minor(outlier_cloud)

    # # paint removal parts
    # epsilon = 0.05
    # points = np.asarray(pcd.points)
    # indices = np.where(np.abs(points[:, 2] < epsilon))[0]
    
    # pcd_in_color = pcd.select_by_index(indices)
    # pcd_in_color.paint_uniform_color([1,0,0])
    # pcd = pcd.select_by_index(indices, invert=True)
    
    # self.visualise(pcd, pcd_in_color, windowName="z index floor")

    # return pcd
downsample_clean(pcd)

In [106]:
# def farthest_point_sampling(point_cloud, num_samples):
#     """
#     Perform farthest point sampling on a point cloud.
    
#     Parameters:
#     - point_cloud: numpy array of shape (N, 3)
#     - num_samples: desired number of points after sampling

#     Returns:
#     - sampled_point_cloud: numpy array of shape (num_samples, 3)
#     """

#     N = point_cloud.shape[0]  # original number of points

#     # Array to hold the indices of the sampled points
#     sampled_indices = np.zeros((num_samples,), dtype=int)

#     # Array to hold the squared distances to the nearest sampled point
#     sq_distances = np.full((N,), np.inf)

#     # Randomly choose the first point
#     sampled_indices[0] = np.random.randint(N)
#     sq_distances[sampled_indices[0]] = 0

#     for i in range(1, num_samples):
#         # Compute squared distances to the latest sampled point
#         new_sq_distances = np.sum((point_cloud - point_cloud[sampled_indices[i - 1]])**2, axis=1)

#         # Update the squared distances to the nearest sampled point
#         np.minimum(sq_distances, new_sq_distances, out=sq_distances)

#         # Choose the point with the greatest squared distance to the nearest sampled point
#         sampled_indices[i] = np.argmax(sq_distances)

#     # Return the sampled points
#     return point_cloud[sampled_indices]

# sampled_points = farthest_point_sampling(np.asarray(pcd.points), num_samples=1000)
# sampled_pcd = o3d.geometry.PointCloud()
# sampled_pcd.points = o3d.utility.Vector3dVector(sampled_points)
# o3d.visualization.draw_geometries([sampled_pcd])

In [62]:
def bounding_box_major(self, pcd):
    '''
    applying bounding box method removing majority of unwanted points
    '''
    
    points = np.asarray(pcd.points)
    
    min_x, min_y, min_z = points.min(axis=0)[:3]
    max_x, max_y, max_z = points.max(axis=0)[:3]

    padding = 0.53
    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(padding*min_x, padding*min_y ,min_z), max_bound=(padding*max_x, padding*max_y, max_z))
    bbox.get_minimal_oriented_bounding_box()
    bbox.color = [1,0,0]

    # Select points within the bounding box
    pcd_in_bbox = pcd.crop(bbox)
    
    return pcd_in_bbox

        
process = PreProcessing()